In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/new-york-city-taxi-fare-prediction/sample_submission.csv
/kaggle/input/new-york-city-taxi-fare-prediction/GCP-Coupons-Instructions.rtf
/kaggle/input/new-york-city-taxi-fare-prediction/train.csv
/kaggle/input/new-york-city-taxi-fare-prediction/test.csv


In [2]:
# train_df = pd.read_csv('/kaggle/input/new-york-city-taxi-fare-prediction/train.csv')
# train_df.head()

In [3]:
# test_df = pd.read_csv('/kaggle/input/new-york-city-taxi-fare-prediction/test.csv')
# test_df.head()

In [4]:
selected_cols = ['fare_amount',
                'pickup_datetime',
                'pickup_longitude',
                'pickup_latitude',
                'dropoff_longitude',
                'dropoff_latitude',
                'passenger_count']

In [5]:
dtypes = {'fare_amount': 'float32',
                'pickup_longitude': 'float32',
                'pickup_latitude': 'float32',
                'dropoff_longitude': 'float32',
                'dropoff_latitude': 'float32',
                'passenger_count': 'uint8'
}

In [6]:
train_df_part = pd.read_csv('/kaggle/input/new-york-city-taxi-fare-prediction/train.csv', 
                           usecols=selected_cols, dtype=dtypes,
                           parse_dates=['pickup_datetime'],
                           nrows=100000)
train_df_part.head()

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,4.5,2009-06-15 17:26:21+00:00,-73.844315,40.721317,-73.841614,40.712276,1
1,16.9,2010-01-05 16:52:16+00:00,-74.016045,40.711304,-73.979271,40.782005,1
2,5.7,2011-08-18 00:35:00+00:00,-73.982735,40.761269,-73.991241,40.750561,2
3,7.7,2012-04-21 04:30:42+00:00,-73.987129,40.733143,-73.991570,40.758091,1
4,5.3,2010-03-09 07:51:00+00:00,-73.968094,40.768009,-73.956657,40.783764,1


load test dataset

In [7]:
test_df = pd.read_csv('/kaggle/input/new-york-city-taxi-fare-prediction/test.csv', 
                           dtype=dtypes,
                           parse_dates=['pickup_datetime'])
test_df.head()

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2015-01-27 13:08:24.0000002,2015-01-27 13:08:24+00:00,-73.973320,40.763805,-73.981430,40.743835,1
1,2015-01-27 13:08:24.0000003,2015-01-27 13:08:24+00:00,-73.986862,40.719383,-73.998886,40.739201,1
2,2011-10-08 11:53:44.0000002,2011-10-08 11:53:44+00:00,-73.982521,40.751259,-73.979652,40.746140,1
3,2012-12-01 21:12:12.0000002,2012-12-01 21:12:12+00:00,-73.981163,40.767807,-73.990448,40.751637,1
4,2012-12-01 21:12:12.0000003,2012-12-01 21:12:12+00:00,-73.966049,40.789776,-73.988564,40.744427,1


# explore the dataset

In [8]:
train_df_part.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype              
---  ------             --------------   -----              
 0   fare_amount        100000 non-null  float32            
 1   pickup_datetime    100000 non-null  datetime64[ns, UTC]
 2   pickup_longitude   100000 non-null  float32            
 3   pickup_latitude    100000 non-null  float32            
 4   dropoff_longitude  100000 non-null  float32            
 5   dropoff_latitude   100000 non-null  float32            
 6   passenger_count    100000 non-null  uint8              
dtypes: datetime64[ns, UTC](1), float32(5), uint8(1)
memory usage: 2.8 MB


In [9]:
train_df_part.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,11.354651,-72.494682,39.914482,-72.490959,39.919056,1.673820
std,9.716777,10.693933,6.225686,10.471387,6.213427,1.300171
min,-44.900002,-736.549988,-74.007668,-84.654243,-74.006378,0.000000
25%,6.000000,-73.992043,40.734997,-73.991219,40.734180,1.000000
50%,8.500000,-73.981789,40.752766,-73.980003,40.753242,1.000000
75%,12.500000,-73.966980,40.767258,-73.963432,40.768166,2.000000
max,200.000000,40.787575,401.083344,40.851028,404.616669,6.000000


In [10]:
df = train_df_part
df['pickup_datetime'].min(), df['pickup_datetime'].max()

(Timestamp('2009-01-01 00:41:00+0000', tz='UTC'),
 Timestamp('2015-06-30 22:54:07+0000', tz='UTC'))

## we could limit ranges to ranges in test dataset

### split to train and validation

In [11]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.2 )

In [12]:
train_df = train_df.dropna()
val_df = val_df.dropna()

In [13]:
X_cols = ['pickup_longitude',
            'pickup_latitude',
            'dropoff_longitude',
            'dropoff_latitude',
            'passenger_count']
y_col = 'fare_amount'

In [14]:
X_train = train_df[X_cols]
y_train = train_df[y_col]

In [15]:
X_train.head()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
2744,-73.998276,40.740547,-74.005135,40.729626,6
1191,-73.976089,40.791340,-73.981438,40.745686,1
81441,-73.984581,40.764889,-73.993797,40.737000,1
14533,-73.996765,40.737579,-73.993752,40.751804,1
33793,-73.997864,40.737675,-73.913055,40.774235,1


In [16]:
y_train.head()

2744      5.5
1191     17.0
81441    16.5
14533     6.5
33793    23.5
Name: fare_amount, dtype: float32

In [17]:
X_val = val_df[X_cols]
y_val = val_df[y_col]

In [18]:
test_df = test_df[X_cols]
test_df.head()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,-73.973320,40.763805,-73.981430,40.743835,1
1,-73.986862,40.719383,-73.998886,40.739201,1
2,-73.982521,40.751259,-73.979652,40.746140,1
3,-73.981163,40.767807,-73.990448,40.751637,1
4,-73.966049,40.789776,-73.988564,40.744427,1


Train Hardcoded and baseline models

In [19]:
class MeanRegressor:
    def fit(self, inputs, targets):
        self.mean = targets.mean()
        
    def predict(self, inputs):
        return np.full(inputs.shape[0], self.mean)

In [20]:
mean_model = MeanRegressor()

In [21]:
mean_model.fit(X_train, y_train)
mean_model.mean

11.354449

In [22]:
train_pred = mean_model.predict(X_train)

In [23]:
val_pred = mean_model.predict(X_val)

In [24]:
from sklearn.metrics import mean_squared_error

In [25]:
def rmse(targets, preds):
    #get root mean squared error
    return mean_squared_error(targets, preds, squared=False)

In [26]:
train_rmse = rmse(y_train, train_pred)
train_rmse

9.749306

In [27]:
val_rmse = rmse(y_val, val_pred)
val_rmse

9.585312

## train and evaluate baseline model

In [28]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression()
reg.fit(X_train, y_train)
train_pred = reg.predict(X_train)
val_pred = reg.predict(X_val)

In [29]:
train_rmse = rmse(y_train, train_pred)
train_rmse

9.747409

In [30]:
val_rmse = rmse(y_val, val_pred)
val_rmse

9.583403

In [31]:
# to submit 
test_pred = reg.predict(test_df)
sub_df = pd.read_csv('/kaggle/input/new-york-city-taxi-fare-prediction/sample_submission.csv')
sub_df

,key,fare_amount
0,2015-01-27 13:08:24.0000002,11.35
1,2015-01-27 13:08:24.0000003,11.35
2,2011-10-08 11:53:44.0000002,11.35
3,2012-12-01 21:12:12.0000002,11.35
4,2012-12-01 21:12:12.0000003,11.35
...,...,...
9909,2015-05-10 12:37:51.0000002,11.35
9910,2015-01-12 17:05:51.0000001,11.35
9911,2015-04-19 20:44:15.0000001,11.35
9912,2015-01-31 01:05:19.0000005,11.35


In [32]:
sub_df['fare_amount'] = test_pred
sub_df

,key,fare_amount
0,2015-01-27 13:08:24.0000002,11.257696
1,2015-01-27 13:08:24.0000003,11.259074
2,2011-10-08 11:53:44.0000002,11.258145
3,2012-12-01 21:12:12.0000002,11.257758
4,2012-12-01 21:12:12.0000003,11.256870
...,...,...
9909,2015-05-10 12:37:51.0000002,11.902442
9910,2015-01-12 17:05:51.0000001,11.902212
9911,2015-04-19 20:44:15.0000001,11.900764
9912,2015-01-31 01:05:19.0000005,11.905018


In [33]:
sub_df.to_csv('linearReg.csv', index=None)

## Reusable functions

In [34]:
def predict_and_submit(model, test_imputs, fname):  
    # to submit 
    test_pred = model.predict(test_imputs)
    sub_df = pd.read_csv('/kaggle/input/new-york-city-taxi-fare-prediction/sample_submission.csv')
    sub_df['fare_amount'] = test_pred
    sub_df.to_csv(fname, index=None)
    return sub_df

##  Feature Engineering

In [35]:
def add_dateparts(df, col):
    df[col +"_year"] = df[col].dt.year
    df[col + "_month"] = df[col].dt.month
    df[col +"_day"] = df[col].dt.day
    df[col + "_weekday"] = df[col].dt.weekday
    df[col+"_hour"] = df[col].dt.hour

In [36]:
col = 'pickup_datetime'
add_dateparts(df, col)

In [37]:
df.head()

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_datetime_year,pickup_datetime_month,pickup_datetime_day,pickup_datetime_weekday,pickup_datetime_hour
0,4.5,2009-06-15 17:26:21+00:00,-73.844315,40.721317,-73.841614,40.712276,1,2009,6,15,0,17
1,16.9,2010-01-05 16:52:16+00:00,-74.016045,40.711304,-73.979271,40.782005,1,2010,1,5,1,16
2,5.7,2011-08-18 00:35:00+00:00,-73.982735,40.761269,-73.991241,40.750561,2,2011,8,18,3,0
3,7.7,2012-04-21 04:30:42+00:00,-73.987129,40.733143,-73.991570,40.758091,1,2012,4,21,5,4
4,5.3,2010-03-09 07:51:00+00:00,-73.968094,40.768009,-73.956657,40.783764,1,2010,3,9,1,7


In [38]:
test_df = pd.read_csv('/kaggle/input/new-york-city-taxi-fare-prediction/test.csv', 
                           dtype=dtypes,
                           parse_dates=['pickup_datetime'])

In [39]:
add_dateparts(test_df, col)
test_df.head()

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_datetime_year,pickup_datetime_month,pickup_datetime_day,pickup_datetime_weekday,pickup_datetime_hour
0,2015-01-27 13:08:24.0000002,2015-01-27 13:08:24+00:00,-73.973320,40.763805,-73.981430,40.743835,1,2015,1,27,1,13
1,2015-01-27 13:08:24.0000003,2015-01-27 13:08:24+00:00,-73.986862,40.719383,-73.998886,40.739201,1,2015,1,27,1,13
2,2011-10-08 11:53:44.0000002,2011-10-08 11:53:44+00:00,-73.982521,40.751259,-73.979652,40.746140,1,2011,10,8,5,11
3,2012-12-01 21:12:12.0000002,2012-12-01 21:12:12+00:00,-73.981163,40.767807,-73.990448,40.751637,1,2012,12,1,5,21
4,2012-12-01 21:12:12.0000003,2012-12-01 21:12:12+00:00,-73.966049,40.789776,-73.988564,40.744427,1,2012,12,1,5,21


In [40]:
test_df.drop(columns ='key', inplace=True)

## Add distance between pickup and dropoff location

#### use haversine formuala

In [41]:
def haversine(lon1, lat1, lon2, lat2):
    """
    calculate the great circle distance between two points
    on the earth (in decimal degrees)
    
    all args must be of equal length 
    works on a list
    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    km = 6367 * c
    return km

In [42]:
lon1 = df['pickup_longitude']
lat1 = df['pickup_latitude']
lon2 = df['dropoff_longitude']
lat2 = df['dropoff_latitude']

df['trip_distance'] = haversine(lon1, lat1, lon2, lat2)

In [43]:
lon1 = test_df['pickup_longitude']
lat1 = test_df['pickup_latitude']
lon2 = test_df['dropoff_longitude']
lat2 = test_df['dropoff_latitude']

test_df['trip_distance'] = haversine(lon1, lat1, lon2, lat2)
test_df.head()

,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_datetime_year,pickup_datetime_month,pickup_datetime_day,pickup_datetime_weekday,pickup_datetime_hour,trip_distance
0,2015-01-27 13:08:24+00:00,-73.973320,40.763805,-73.981430,40.743835,1,2015,1,27,1,13,2.321899
1,2015-01-27 13:08:24+00:00,-73.986862,40.719383,-73.998886,40.739201,1,2015,1,27,1,13,2.423777
2,2011-10-08 11:53:44+00:00,-73.982521,40.751259,-73.979652,40.746140,1,2011,10,8,5,11,0.618015
3,2012-12-01 21:12:12+00:00,-73.981163,40.767807,-73.990448,40.751637,1,2012,12,1,5,21,1.959681
4,2012-12-01 21:12:12+00:00,-73.966049,40.789776,-73.988564,40.744427,1,2012,12,1,5,21,5.383829


In [44]:
train_df, val_df = train_test_split(df, test_size=0.2 )

In [45]:
train_df.head(4)

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_datetime_year,pickup_datetime_month,pickup_datetime_day,pickup_datetime_weekday,pickup_datetime_hour,trip_distance
22816,7.1,2009-02-20 16:25:59+00:00,-73.972496,40.746262,-73.985474,40.734310,1,2009,2,20,4,16,1.719931
93140,8.1,2010-03-19 09:35:32+00:00,-73.986176,40.728127,-73.997292,40.746616,1,2010,3,19,4,9,2.257897
71262,8.1,2011-10-05 23:49:40+00:00,-73.994095,40.731213,-73.986954,40.754559,2,2011,10,5,2,23,2.663101
71736,5.0,2015-06-02 10:55:08+00:00,-73.975151,40.758755,-73.980240,40.752522,1,2015,6,2,1,10,0.814344


## add distance from popular landmarks
#### add distance from drop location to
1- JFK, LAG, EWR airports
2- Times Square
3- Met Museum
4- World Trade Center


In [46]:
jfk_lonlat = -73.7781, 40.6413
lga_lonlat = -73.8740, 40.7769
ewr_lonlat = -74.1745, 40.6895
met_lonlat = -73.9632, 40.7794
wtc_lonlat = -74.0099, 40.7126

In [47]:
def add_landmark_dropoff_distance(df, landmark_name, landmark_lonlat):
    lon, lat = landmark_lonlat
    df[landmark_name + '_drop_distance'] = haversine(lon, lat, df['dropoff_longitude'], df['dropoff_latitude'])

In [48]:
for a_df in [train_df, val_df, test_df]:
    for name, lonlat in [('jfk', jfk_lonlat), ('lga', lga_lonlat), ('ewr', ewr_lonlat), ('met', met_lonlat), ('wtc', wtc_lonlat)]:
        add_landmark_dropoff_distance(a_df, name, lonlat)

We'll use the following ranges:

    fare_amount: 1to500
    longitudes: -75 to -72
    latitudes: 40 to 42
    passenger_count: 1 to 6

In [49]:
df.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_datetime_year,pickup_datetime_month,pickup_datetime_day,pickup_datetime_weekday,pickup_datetime_hour,trip_distance
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.00000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,11.354651,-72.494682,39.914482,-72.490959,39.919056,1.673820,2011.74389,6.261230,15.664270,3.036180,13.495110,18.580544
std,9.716777,10.693933,6.225686,10.471387,6.213427,1.300171,1.86602,3.450569,8.677631,1.949484,6.507434,356.537933
min,-44.900002,-736.549988,-74.007668,-84.654243,-74.006378,0.000000,2009.00000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,6.000000,-73.992043,40.734997,-73.991219,40.734180,1.000000,2010.00000,3.000000,8.000000,1.000000,9.000000,1.213685
50%,8.500000,-73.981789,40.752766,-73.980003,40.753242,1.000000,2012.00000,6.000000,16.000000,3.000000,14.000000,2.117939
75%,12.500000,-73.966980,40.767258,-73.963432,40.768166,2.000000,2013.00000,9.000000,23.000000,5.000000,19.000000,3.885313
max,200.000000,40.787575,401.083344,40.851028,404.616669,6.000000,2015.00000,12.000000,31.000000,6.000000,23.000000,8669.175781


In [50]:
def remove_outliers(df):
    return df[(df['fare_amount'] >= 1.) & 
              (df['fare_amount'] <= 500.) &
              (df['pickup_longitude'] >= -75) & 
              (df['pickup_longitude'] <= -72) & 
              (df['dropoff_longitude'] >= -75) & 
              (df['dropoff_longitude'] <= -72) & 
              (df['pickup_latitude'] >= 40) & 
              (df['pickup_latitude'] <= 42) & 
              (df['dropoff_latitude'] >=40) & 
              (df['dropoff_latitude'] <= 42) & 
              (df['passenger_count'] >= 1) & 
              (df['passenger_count'] <= 6)]

In [51]:
train_df = remove_outliers(train_df)

In [52]:
val_df = remove_outliers(val_df)

### we could scale data later

## train models

#### Ridge Regression

In [53]:
X_train = train_df.drop(columns=['fare_amount', 'pickup_datetime'])
y_train = train_df['fare_amount']

In [54]:
X_val = val_df.drop(columns=['fare_amount', 'pickup_datetime'])
y_val = val_df['fare_amount']

In [55]:
test_df=test_df.drop(columns = 'pickup_datetime')

In [56]:
test_df.head(3)

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_datetime_year,pickup_datetime_month,pickup_datetime_day,pickup_datetime_weekday,pickup_datetime_hour,trip_distance,jfk_drop_distance,lga_drop_distance,ewr_drop_distance,met_drop_distance,wtc_drop_distance
0,-73.973320,40.763805,-73.981430,40.743835,1,2015,1,27,1,13,2.321899,20.574911,9.760167,17.346842,4.239343,4.218709
1,-73.986862,40.719383,-73.998886,40.739201,1,2015,1,27,1,13,2.423777,21.550976,11.315990,15.789623,5.382879,3.098136
2,-73.982521,40.751259,-73.979652,40.746140,1,2011,10,8,5,11,0.618015,20.594069,9.526829,17.576965,3.946721,4.514503


In [57]:
def evaluate(model , train_inputs, train_targets, val_inputs, val_targets):
    train_pred = model.predict(train_inputs)
    train_rmse = rmse(train_targets, train_pred)
    
    val_pred = model.predict(val_inputs)
    val_rmse = rmse(val_targets, val_pred)
    
    return train_rmse, val_rmse, train_pred, val_pred

In [58]:
from sklearn.linear_model import Ridge

In [59]:
model1 = Ridge(alpha=0.9)
model1.fit(X_train, y_train)

Ridge(alpha=0.9)

In [60]:
evaluate(model1, X_train, y_train, X_val, y_val)

(5.326117523346616,
 5.611531415462144,
 array([ 6.32861593,  8.37121456,  9.82042067, ...,  7.30930831,
        10.35476797, 23.6933982 ]),
 array([ 8.59818019, 10.12984928,  7.20813377, ...,  7.51492656,
         5.10248606, 22.24490651]))

In [61]:
predict_and_submit(model1, test_df, 'Ridge.csv')

,key,fare_amount
0,2015-01-27 13:08:24.0000002,10.187471
1,2015-01-27 13:08:24.0000003,11.502309
2,2011-10-08 11:53:44.0000002,5.707271
3,2012-12-01 21:12:12.0000002,8.710552
4,2012-12-01 21:12:12.0000003,13.953351
...,...,...
9909,2015-05-10 12:37:51.0000002,9.010244
9910,2015-01-12 17:05:51.0000001,11.234315
9911,2015-04-19 20:44:15.0000001,47.402014
9912,2015-01-31 01:05:19.0000005,21.665360


# Random Forest

In [62]:
from sklearn.ensemble import RandomForestRegressor

In [63]:
model2 = RandomForestRegressor(max_depth=10, n_jobs=-1, random_state=42, n_estimators=50)

In [64]:
model2.fit(X_train, y_train)

RandomForestRegressor(max_depth=10, n_estimators=50, n_jobs=-1, random_state=42)

In [65]:
evaluate(model2, X_train, y_train, X_val, y_val)

(3.2799850492456657,
 4.369673952581463,
 array([ 7.12605579,  8.16815145,  8.81286858, ...,  5.49895337,
         9.18569119, 29.85372471]),
 array([ 7.31501694, 11.06197729,  5.36252115, ...,  7.16691342,
         5.24534503, 33.52712937]))

In [66]:
predict_and_submit(model2, test_df, 'rf_submission.csv')

,key,fare_amount
0,2015-01-27 13:08:24.0000002,10.524685
1,2015-01-27 13:08:24.0000003,10.525550
2,2011-10-08 11:53:44.0000002,4.860634
3,2012-12-01 21:12:12.0000002,8.209483
4,2012-12-01 21:12:12.0000003,15.305116
...,...,...
9909,2015-05-10 12:37:51.0000002,9.251399
9910,2015-01-12 17:05:51.0000001,12.821020
9911,2015-04-19 20:44:15.0000001,55.335369
9912,2015-01-31 01:05:19.0000005,21.439991


In [67]:
from sklearn.model_selection import GridSearchCV

In [68]:
model2

RandomForestRegressor(max_depth=10, n_estimators=50, n_jobs=-1, random_state=42)

In [69]:
parameters = {'n_estimators':(50, 80), 'max_depth':[10, 20]}
model_2 = RandomForestRegressor()
model2_GS= GridSearchCV(model_2, parameters)

In [70]:
model2_GS.fit(X_train, y_train)

GridSearchCV(estimator=RandomForestRegressor(),
             param_grid={'max_depth': [10, 20], 'n_estimators': (50, 80)})

In [71]:
model2_GS.best_params_

{'max_depth': 20, 'n_estimators': 80}

In [72]:
evaluate(model2_GS, X_train, y_train, X_val, y_val)

(1.7998621361114802,
 4.326903258528662,
 array([ 6.5698331 ,  8.26764608,  8.39170951, ...,  5.46054232,
         8.3994305 , 33.50113948]),
 array([ 7.86640848, 11.4862139 ,  5.25759671, ...,  6.71560056,
         4.82469452, 33.17725115]))

In [73]:
predict_and_submit(model2, test_df, 'rf_gridsearch.csv')

,key,fare_amount
0,2015-01-27 13:08:24.0000002,10.524685
1,2015-01-27 13:08:24.0000003,10.525550
2,2011-10-08 11:53:44.0000002,4.860634
3,2012-12-01 21:12:12.0000002,8.209483
4,2012-12-01 21:12:12.0000003,15.305116
...,...,...
9909,2015-05-10 12:37:51.0000002,9.251399
9910,2015-01-12 17:05:51.0000001,12.821020
9911,2015-04-19 20:44:15.0000001,55.335369
9912,2015-01-31 01:05:19.0000005,21.439991
